# Compare the performances of a model in onnx mode or pth mode

In [ ]:
import os
from PIL import Image
from torch import Tensor
import numpy as np
import torch
import torch.onnx
import torchvision.models as Model
from torchvision import transforms
import onnx
import onnxruntime
from scipy.special import softmax
from prepare_data import ConvertRgb, Rescale, RandomPad

MODEL_NAME = 'EffB4'

NETS = {
    'EffB0': {'input_size': 224, 'model': Model.efficientnet_b0},
    'EffB1': {'input_size': 240, 'model': Model.efficientnet_b1},
    'EffB2': {'input_size': 288, 'model': Model.efficientnet_b2},
    'EffB3': {'input_size': 300, 'model': Model.efficientnet_b3},
    'EffB4': {'input_size': 380, 'model': Model.efficientnet_b4},
    'EffB5': {'input_size': 456, 'model': Model.efficientnet_b5},
    'EffB6': {'input_size': 528, 'model': Model.efficientnet_b6},
    'EffB7': {'input_size': 600, 'model': Model.efficientnet_b7},
    'Res18': {'input_size': 224, 'model': Model.resnet18},
    'Dense169': {'input_size': 224, 'model': Model.densenet169},
    'Dense201': {'input_size': 224, 'model': Model.densenet201}
    }

MODEL_TORCH = NETS[MODEL_NAME]['model']
INPUT_SIZE = NETS[MODEL_NAME]['input_size']

CLASSES = ['autre_epaule', 'autre_pistolet', 'epaule_a_levier_sous_garde',
        'epaule_a_percussion_silex', 'epaule_a_pompe', 'epaule_a_un_coup', 'epaule_a_verrou',
        'pistolet_a_percussion_silex', 'pistolet_semi_auto_moderne', 'revolver']

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [ ]:
loader =  transforms.Compose([
            ConvertRgb(),
            Rescale(INPUT_SIZE),
            RandomPad(INPUT_SIZE),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

def build_model(model: Model) -> Model:
    # freeze first layers
    for param in model.parameters():
        param.requires_grad = False
    # Parameters of newly constructed modules have requires_grad=True by default
    num_ftrs = model.classifier[1].in_features
    # to try later : add batch normalization and dropout
    model.classifier[1] = torch.nn.Linear(num_ftrs, len(CLASSES))
    model = model.to(device)
    return model


def load_model_inference(state_dict_path: str) -> Model:
    model = build_model(MODEL_TORCH())
    # Initialize model with the pretrained weights
    model.load_state_dict(torch.load(state_dict_path, map_location=device)['model_state_dict'])
    model.to(device)
    # set the model to inference mode
    model.eval()
    return model


ort_session = onnxruntime.InferenceSession('models/EffB4_2022-03-02_08.onnx')
model = load_model_inference('models/EffB4_2022-03-02_08.pth')


def test_image(path):
    im = Image.open(path)
    image = loader(im).float()
    image = image.unsqueeze(0).to(device)
    # ort_inputs = {ort_session.get_inputs()[0].name: image.detach().cpu().numpy()}
    # output = ort_session.run(None, ort_inputs)[0]
    # probs = softmax(output, axis=1)[0]
    output = model(image)
    probs = torch.nn.functional.softmax(output, dim=1).detach().numpy()[0]
    res = [(CLASSES[i], round(probs[i]*100,2)) for i in range(len(CLASSES))]
    res.sort(key=lambda x:x[1], reverse=True)
    # display(im.resize((300,int(300*im.size[1]/im.size[0])))) # display image in notebook
    return res


In [ ]:
import sys, os, glob, time

def test_folder():
    moy_time = 0
    list_imgs = glob.glob(os.path.join(sys.argv[1], '*'))
    for path in list_imgs:
        t = time.time()
        print(test_image(path)[0], time.time()-t)
        moy_time += time.time()-t
    print("Avg time = ", round(moy_time / len(list_imgs), 2))